### 1. RI sample

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sample = ["c_2","c_7","c_20","c_34","c_39","c_45","c_51","c_3","c_4","c_36"]
gene= ['HSPB1', 'LTB', 'FTL', 'VIM', 'BLK', 'IGLC1', 'C7', 'IGHG1', 'FCRL1', 
       'PTGDS', 'IGHG3', 'IGHA1', 'FN1', 'IGLV3-1', 'ACTB', 'BANK1', 'MT2A', 
       'CCL19', 'IGHM', 'CD37']
#add color bar
def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mpl.transforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax
for s in range(len(sample)):
    img = []
    #load expression matrix
    path = "result/"+sample[s]+"_matrix.csv"
    data1 = pd.read_csv(path)
    l_gene = data1["Unnamed: 0"]
    #adjust barcode
    bar = data1.columns[1:]
    l_bar=[]
    for j in bar:
        j=str(j).replace("X","")
        l_bar.append(j)
    a = data1.shape[0]
    b = data1.shape[1]-1
    data1 = np.array(data1[bar])
    #normalization
    X_scaler = scaler.fit(data1)                                   
    data1 = X_scaler.transform(data1)                                    
    data1 = pd.DataFrame(data1, columns = l_bar, index = l_gene)
    #load position matrix
    path2 = 'position/'+sample[s]+'_positions_list.csv'
    data2 = pd.read_csv(path2)
    data2 = data2[['barcodes','X','Y']]
    data2 = pd.DataFrame(data2.values.T,index = ['barcodes','X','Y'])
    l_barcodes2 = data2.loc['barcodes'].tolist()
    data2 = pd.DataFrame(data2.values,columns = data2.loc['barcodes'])
    data2.index = ['barcodes','X','Y']
    #connect gene expression and position
    for m in range(len(gene)):
        sub_data1 = data1.loc[gene[m]]
        l_barcodes1 = data1.columns.values.tolist()
        D_position = {}
        L_positionX = []
        L_positionY = []
        l_barcodes4 = []
        for i in l_barcodes2:
            x = data2.loc['X',i]
            y = data2.loc['Y',i]
            i = i.replace('-1','')
            D_position[i] = (x,y)
            L_positionX.append(x)
            L_positionY.append(y)
            l_barcodes4.append(i)
        D_Bcell = {}
        l_barcodes3 = []
        for j in l_barcodes1:
            bcell = sub_data1.loc[j]
            j = j.replace('.1','')
            D_Bcell[j] = bcell
            l_barcodes3.append(j)
        L_cellvalues = []
        count = 0
        for k in l_barcodes4:
            if k in l_barcodes3:
                L_cellvalues.append(D_Bcell[k])
            else:
                L_cellvalues.append(0.0)
        results = np.array([L_positionX,L_positionY,L_cellvalues])
        results = pd.DataFrame(results)
        results = pd.DataFrame(results.values,columns = l_barcodes4)
        results.index = ['x','y','gene']
        L_cellvalues = np.array(L_cellvalues)
        L_cellvalues = L_cellvalues.reshape(-1, 1)
        im = L_cellvalues
        img.append(im)
    #plot immunoglobulin genes
    ig_gene= ['IGLC1', 'IGHG1','IGHG3', 'IGHA1','IGLV3-1','IGHM']
    fig, ax = plt.subplots(2, 3, figsize=(18,12))
    import matplotlib as mpl
    cmp = mpl.colors.ListedColormap(['black','cornflowerblue','lightgreen','yellow','orange','red'])
    norm = mpl.colors.BoundaryNorm([0,0.001,0.05,0.1,0.15,0.2,0.25], cmp.N)
    im = mpl.cm.ScalarMappable(norm=norm, cmap=cmp)
    plt.axis([-0.1, 78, -0.1, 127])
    for i in range(len(ig_gene)):
        x = i//3
        y = i%3
        name = ig_gene[i]
        ax[x,y].set_title(name, fontsize=30)
        ax[x,y].get_xaxis().set_visible(False)
        ax[x,y].get_yaxis().set_visible(False)
        k = gene.index(name)
        ax[x,y].scatter(np.array(L_positionX),np.array(L_positionY), c=img[k], edgecolors='none', cmap=cmp, norm=norm,alpha=0.7)
        cax = add_right_cax(ax[x,2], pad=0.02, width=0.02)
        plt.colorbar(im, cax =cax)
    plt.savefig(sample[s]+'Figure5_h_IG.eps',dpi=600,bbox_inches='tight')#保存文件在指定文件夹下很方便
    plt.show()
    plt.close()
    #plot all biomarkers
    gene= ['HSPB1', 'LTB', 'FTL', 'VIM', 'BLK', 'IGLC1', 'C7', 'IGHG1', 'FCRL1', 
       'PTGDS', 'IGHG3', 'IGHA1', 'FN1', 'IGLV3-1', 'ACTB', 'BANK1', 'MT2A', 
       'CCL19', 'IGHM', 'CD37']
    fig, ax = plt.subplots(4, 5, figsize=(30,24))
    cmp = mpl.colors.ListedColormap(['black','cornflowerblue','lightgreen','yellow','orange','red'])
    norm = mpl.colors.BoundaryNorm([0,0.001,0.1,0.2,0.3,0.4,0.5], cmp.N)
    im = mpl.cm.ScalarMappable(norm=norm, cmap=cmp)
    plt.axis([-0.1, 78, -0.1, 127])
    for i in range(len(gene)):
        x = i//5
        y = i%5
        name = gene[i]
        ax[x,y].set_title(name, fontsize=30)
        ax[x,y].get_xaxis().set_visible(False)
        ax[x,y].get_yaxis().set_visible(False)
        ax[x,y].scatter(np.array(L_positionX),np.array(L_positionY), c=img[i], edgecolors='none', cmap=cmp, norm=norm,alpha=0.7)
        cax = add_right_cax(ax[x,4], pad=0.02, width=0.02)
        plt.colorbar(im, cax =cax)
    plt.savefig(sample[s]+'Figure5_ALL.jpg',dpi=600,bbox_inches='tight')#保存文件在指定文件夹下很方便
    plt.show()
    plt.close()

### 2. NRI sample

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sample = ['a_3','a_15','b_1','b_18']
gene= ['TGFBI', 'TPT1', 'FTL', 'IGHG4', 'IGKC', 'IGLC1', 'EEF1A1',
        'IGLC3', 'IGHGP', 'IGLC2', 'DCN', 'IGHG2', 'RPS27', 'VIM',
        'IGHG3', 'FTH1', 'IGHA1']
def add_right_cax(ax, pad, width):
    axpos = ax.get_position()
    caxpos = mpl.transforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax
for s in range(len(sample)):
    img = []
    path = "result/"+sample[s]+"_matrix.csv"
    data1 = pd.read_csv(path)
    l_gene = data1["Unnamed: 0"]
    bar = data1.columns[1:]
    l_bar=[]
    for j in bar:
        j=str(j).replace("X","")
        l_bar.append(j)
    a = data1.shape[0]
    b = data1.shape[1]-1
    data1 = np.array(data1[bar])
    X_scaler = scaler.fit(data1)                                   
    data1 = X_scaler.transform(data1)                                    
    data1 = pd.DataFrame(data1, columns = l_bar, index = l_gene)
    #data1.index = list(data1['Unnamed: 0'])
    #data1 = data1.drop(['Unnamed: 0'],axis=1)
    path2 = 'position/'+sample[s]+'_positions_list.csv'
    data2 = pd.read_csv(path2)
    data2 = data2[['barcodes','X','Y']]
    data2 = pd.DataFrame(data2.values.T,index = ['barcodes','X','Y'])
    l_barcodes2 = data2.loc['barcodes'].tolist()
    data2 = pd.DataFrame(data2.values,columns = data2.loc['barcodes'])
    data2.index = ['barcodes','X','Y']
    for m in range(len(gene)):
        sub_data1 = data1.loc[gene[m]]
        l_barcodes1 = data1.columns.values.tolist()
        D_position = {}
        L_positionX = []
        L_positionY = []
        l_barcodes4 = []
        for i in l_barcodes2:
            x = data2.loc['X',i]
            y = data2.loc['Y',i]
            i = i.replace('-1','')
            D_position[i] = (x,y)
            L_positionX.append(x)
            L_positionY.append(y)
            l_barcodes4.append(i)
        D_Bcell = {}
        l_barcodes3 = []
        for j in l_barcodes1:
            bcell = sub_data1.loc[j]
            j = j.replace('.1','')
            D_Bcell[j] = bcell
            l_barcodes3.append(j)
        L_cellvalues = []
        count = 0
        for k in l_barcodes4:
            if k in l_barcodes3:
                L_cellvalues.append(D_Bcell[k])
            else:
                L_cellvalues.append(0.0)
        results = np.array([L_positionX,L_positionY,L_cellvalues])
        results = pd.DataFrame(results)
        results = pd.DataFrame(results.values,columns = l_barcodes4)
        results.index = ['x','y','gene']
        L_cellvalues = np.array(L_cellvalues)
        L_cellvalues = L_cellvalues.reshape(-1, 1)
        im = L_cellvalues
        img.append(im)
    ig_gene= [ 'IGHG4', 'IGKC', 'IGLC1','IGLC3', 'IGHGP', 'IGLC2', 'IGHG2','IGHG3','IGHA1']
    fig, ax = plt.subplots(3, 3, figsize=(18,18))
    import matplotlib as mpl
    cmp = mpl.colors.ListedColormap(['black','cornflowerblue','lightgreen','yellow','orange','red'])
    norm = mpl.colors.BoundaryNorm([0,0.001,0.05,0.1,0.15,0.2,0.25], cmp.N)
    im = mpl.cm.ScalarMappable(norm=norm, cmap=cmp)
    #plt.xlabel("x_position", fontsize=14)
    #plt.ylabel("y_position", fontsize=14)
    plt.axis([-0.1, 78, -0.1, 127])
    for i in range(len(ig_gene)):
        x = i//3
        y = i%3
        name = ig_gene[i]
        ax[x,y].set_title(name, fontsize=30)
        ax[x,y].get_xaxis().set_visible(False)
        ax[x,y].get_yaxis().set_visible(False)
        k = gene.index(name)
        ax[x,y].scatter(np.array(L_positionX),np.array(L_positionY), c=img[k], edgecolors='none', cmap=cmp, norm=norm,alpha=0.7)
        cax = add_right_cax(ax[x,2], pad=0.02, width=0.02)
        plt.colorbar(im, cax =cax)
    plt.savefig(sample[s]+'Figure5_h_IG.eps',dpi=600,bbox_inches='tight')#保存文件在指定文件夹下很方便
    plt.show()
    plt.close()
    gene= ['TGFBI', 'TPT1', 'FTL', 'IGHG4', 'IGKC', 'IGLC1', 'EEF1A1',
           'IGLC3', 'IGHGP', 'IGLC2', 'DCN', 'IGHG2', 'RPS27', 'VIM',
           'IGHG3', 'FTH1', 'IGHA1']
    fig, ax = plt.subplots(4, 5, figsize=(30,24))
    cmp = mpl.colors.ListedColormap(['black','cornflowerblue','lightgreen','yellow','orange','red'])
    norm = mpl.colors.BoundaryNorm([0,0.001,0.1,0.2,0.3,0.4,0.5], cmp.N)
    im = mpl.cm.ScalarMappable(norm=norm, cmap=cmp)
    #plt.xlabel("x_position", fontsize=14)
    #plt.ylabel("y_position", fontsize=14)
    plt.axis([-0.1, 78, -0.1, 127])
    for i in range(len(gene)):
         #s为点的大小
        x = i//5
        y = i%5
        name = gene[i]
        ax[x,y].set_title(name, fontsize=30)
        ax[x,y].get_xaxis().set_visible(False)
        ax[x,y].get_yaxis().set_visible(False)
        ax[x,y].scatter(np.array(L_positionX),np.array(L_positionY), c=img[i], edgecolors='none', cmap=cmp, norm=norm,alpha=0.7)
        cax = add_right_cax(ax[x,4], pad=0.02, width=0.02)
        plt.colorbar(im, cax =cax)
    plt.savefig(sample[s]+'Figure5_ALL.jpg',dpi=600,bbox_inches='tight')#保存文件在指定文件夹下很方便
    plt.show()
    plt.close()